In [1]:
from sqlalchemy import types
from StringIO import StringIO
import sqlite3
import CleanData as cd
import numpy as np
import pandas as pd
import re

# Clean Data

In [2]:
title = 'Roman_Imperial_Coinage'

In [3]:
def cleanMaterials(materialCol):
    '''
    Takes in a string definiing the material column then cleans the material descriptions. 
    
    Currently, it cleans by:
        * Making all the letters to lowercase
    '''
    
    def cleanMat(row):
        material = row[materialCol]
        
        material = material.lower()
        
        return material
    
    return cleanMat

def splitDescription(descriptionCol, side):
    '''
    Takes in a string defining the description column then splits the description 
    to obverse and reverse specified by SIDE
    '''
    
    def splitDesc(row):
        desc = row[descriptionCol]
        result = re.split(r'(\(obverse\)|\(reverse\))', desc)
        if side == 'obverse' and len(result) >= 2:
            return result[2].strip()
        elif side == 'reverse' and len(result) >= 4:
            return result[4].strip()
        else:
            return ''
    
    return splitDesc

### British Museum

In [4]:
# Read in data and convert to appropriate types
bm_df = pd.read_csv('../Data/BM_'+title+'.csv',
                 converters={"Authority": cd.stringToList(), 'Associated names': cd.stringToList(), 
                             'Subjects': cd.stringToList(), 'Inscriptions': cd.stringToListofDicts('|', ';', ':')})
bm_df = pd.concat([bm_df, cd.cleanInscriptions(bm_df)], axis=1)
bm_df = bm_df.drop('Unnamed: 0', 1)
bm_df = bm_df.replace(np.nan, '', regex=True)
bm_df['Denomination'] = bm_df.apply(lambda row: cd.cleanDenomination(row['Denomination']), axis=1)
bm_df = bm_df.rename(columns = {'Culture/period':'Culture'})
bm_df = bm_df[~bm_df["Curator's comments"].str.contains('forgery')]
bm_df['Materials'] = bm_df.apply(cleanMaterials("Materials"), axis=1)
bm_df['Obverse type'] = bm_df.apply(splitDescription('Description', 'obverse'), axis=1)
bm_df['Reverse type'] = bm_df.apply(splitDescription('Description', 'reverse'), axis=1)

KeyboardInterrupt: 

In [ ]:
bm_df.tail()

In [ ]:
# Clean data to only retain relevant information
mask = ((bm_df['Object type'] == 'coin ') & (bm_df['Date'].str.find('stC') == -1))
filtered = bm_df[mask]

lists = ['Authority', 'Subjects', 'Associated names', 'Obverse legend', 'Reverse legend']
strings = ['Museum number', 'Denomination', 'Description', 'State', 'Culture', 'Materials', 
            'Curator\'s comments', 'Bibliography', 'Object type', "Production place", 'Obverse type', 'Reverse type']
floats = ['Weight (g)']
dates = ['Date']
redundant_notes = ['Production place', 'Denomination']
do_nothing = ['url', 'Inscriptions']
duplicate_cols = ['Materials', 'Denomination', 'Subjects', 'Production place', 'Date']

cleaned_bm = cd.cleanDF(filtered, lists, strings, floats, dates, redundant_notes, do_nothing, duplicate_cols)
cleaned_bm = cleaned_bm.drop('Inscriptions', 1)
cleaned_bm['Source'] = 'British Museum'
#cleaned_bm.head()

In [ ]:
cleaned_bm.tail()

In [ ]:
# Convert everything to strings and integers to put into SQL db
lists = ['Authority', 'Subjects', 'Associated names', 'Obverse legend', 'Reverse legend', 'Date']
for col in lists:
    cleaned_bm[col] = cleaned_bm[col].apply(cd.listToString())

columns = [u'Associated names', u'Authority', u'Bibliography', u'Culture',
       u'Curator\'s comments', u'Date', u'Denomination', u'Description',
       u'Materials', u'Museum number', u'Object type', u'Obverse legend', 'Obverse type',
       u'Production place', u'Reverse legend', 'Reverse type', u'State', u'Subjects',
       u'url', u'Source']

for col in columns:
    cleaned_bm[col] = cleaned_bm[col].apply(lambda x: x.decode('utf-8'))

cleaned_bm.columns = ['associatedNames', 'authority', 'bibliography', 'culture',
                       'curatorComment', 'date', 'denomination', 'description',
                       'materials', 'museumNumber', 'objectType', 'obverseLegend', 'obverseType',
                       'mint', 'reverseLegend', 'reverseType', 'state', 'subjects',
                       'weight', 'url', 'source']

cleaned_bm['startDate'] = cleaned_bm.apply(lambda row: row['date'].split(",")[0], axis=1)
cleaned_bm['endDate'] = cleaned_bm.apply(lambda row: row['date'].split(",")[1] 
                                                 if len(row['date'].split(",")) > 1 
                                                 else row['date'].split(",")[0], 
                                         axis=1)

In [ ]:
cleaned_bm.tail()

### American Numismatic Society

In [ ]:
ans_df = pd.read_csv('../Data/ANS_'+title+'.csv',
                    converters={"Authority": cd.stringToList('|'), 'Year': cd.stringToList('|'),
                               'Issuer': cd.stringToList('|'), 'Portrait': cd.stringToList('|'),
                               'Reference': cd.stringToList('|')})
ans_df = ans_df.replace(np.nan, '', regex=True)
ans_df['Description'] = ans_df.apply(lambda row: cd.makeDescription(
                                                                    row['Material'], 
                                                                    row['Manufacture'], 
                                                                    row['Obverse Type'], 
                                                                    row['Reverse Type']
                                                                   ), axis=1)
ans_df['DupCheck'] = ans_df.apply(lambda row: cd.makeDupCheckCol(
                                                                 row['Material'], 
                                                                 row['Denomination'], 
                                                                 row['Portrait'], 
                                                                 row['Mint'],
                                                                 row['Year']
                                                                ), axis=1)
# Clean denomination
ans_df['Denomination'] = ans_df.apply(lambda row: cd.cleanDenomination(row['Denomination']), axis=1)
ans_df = ans_df[~ans_df["Material"].str.contains("Steel")]
ans_df['Material'] = ans_df.apply(cleanMaterials("Material"), axis=1)
ans_df.tail()

In [ ]:
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
strings = ['Deity', 'Denomination', 'Mint', 'Description', 'Obverse Legend',
           'Reverse Legend', 'Material', 'Obverse Type', 'Reverse Type']
floats = []
dates = []
redundant_notes = []
do_nothing = ['URI', 'DupCheck']
duplicate_cols = 'DupCheck'

cleaned_ans = cd.cleanDF(ans_df, lists, strings, floats, dates, redundant_notes, do_nothing, 
                              duplicate_cols, production_place='Mint')
cleaned_ans['Source'] = 'American Numismatic Society'
cleaned_ans.drop(['DupCheck'], axis=1, inplace=True)
cleaned_ans.tail()
cleaned_ans.columns

In [ ]:
# Convert everything to strings and integers to put into SQL db
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
for col in lists:
    cleaned_ans[col] = cleaned_ans[col].apply(cd.listToString())
    
# Get start and end date from date/year column
cleaned_ans['StartDate'] = cleaned_ans.apply(lambda row: row['Year'].split(",")[0], axis=1)
cleaned_ans['EndDate'] = cleaned_ans.apply(lambda row: row['Year'].split(",")[1] 
                                                 if len(row['Year'].split(",")) > 1 
                                                 else row['Year'].split(",")[0], 
                                         axis=1)

# Reorder columns
columns = [u'Authority', u'Deity', u'Denomination', u'Description', u'Issuer',
       'Material', u'Mint', u'Obverse Legend', 'Obverse Type', u'Portrait', u'Reference',
       u'Reverse Legend', 'Reverse Type', u'URI', u'Year', u'Source', 'StartDate', 'EndDate']

cleaned_ans.columns = columns

for col in columns:
    cleaned_ans[col] = cleaned_ans[col].apply(lambda x: x.decode('utf-8') if type(x) is not float else x)

In [ ]:
cleaned_ans.tail()

# OCRE

In [ ]:
ocre_df = pd.read_csv('../Data/OCRE.csv',
                    converters={"Authority": cd.stringToList('|'), 'Year': cd.stringToList('|'),
                               'Issuer': cd.stringToList('|'), 'Portrait': cd.stringToList('|'),
                               'Reference': cd.stringToList('|')})
ocre_df = ocre_df.replace(np.nan, '', regex=True)
ocre_df['Description'] = ocre_df.apply(lambda row: cd.makeDescription(
                                                                    row['Material'], 
                                                                    row['Manufacture'], 
                                                                    row['Obverse Type'], 
                                                                    row['Reverse Type']
                                                                   ), axis=1)
ocre_df['DupCheck'] = ocre_df.apply(lambda row: cd.makeDupCheckCol(
                                                                 row['Material'], 
                                                                 row['Denomination'], 
                                                                 row['Portrait'], 
                                                                 row['Mint'],
                                                                 row['Year']
                                                                ), axis=1)
# Clean denomination
ocre_df['Denomination'] = ocre_df.apply(lambda row: cd.cleanDenomination(row['Denomination']), axis=1)
ocre_df['Material'] = ocre_df.apply(cleanMaterials("Material"), axis=1)
ocre_df.tail()

In [ ]:
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
strings = ['Deity', 'Denomination', 'Mint', 'Description', 'Obverse Legend',
           'Reverse Legend', 'Material', 'Obverse Type', 'Reverse Type']
floats = []
dates = []
redundant_notes = []
do_nothing = ['URI', 'DupCheck']
duplicate_cols = 'DupCheck'

cleaned_ocre = cd.cleanDF(ocre_df, lists, strings, floats, dates, redundant_notes, do_nothing, 
                              duplicate_cols, production_place='Mint')
cleaned_ocre['Source'] = 'OCRE'
cleaned_ocre.drop(['DupCheck'], axis=1, inplace=True)
cleaned_ocre.tail()
cleaned_ocre.columns

In [ ]:
# Convert everything to strings and integers to put into SQL db
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
for col in lists:
    cleaned_ocre[col] = cleaned_ocre[col].apply(cd.listToString())
    
# Get start and end date from date/year column
cleaned_ocre['StartDate'] = cleaned_ocre.apply(lambda row: row['Year'].split(",")[0], axis=1)
cleaned_ocre['EndDate'] = cleaned_ocre.apply(lambda row: row['Year'].split(",")[1] 
                                                 if len(row['Year'].split(",")) > 1 
                                                 else row['Year'].split(",")[0], 
                                         axis=1)

# Reorder columns
columns = [u'Authority', u'Deity', u'Denomination', u'Description', u'Issuer',
       'Material', u'Mint', u'Obverse Legend', 'Obverse Type', u'Portrait', u'Reference',
       u'Reverse Legend', 'Reverse Type', u'URI', u'Year', u'Source', 'StartDate', 'EndDate']

cleaned_ocre.columns = columns

for col in columns:
    cleaned_ocre[col] = cleaned_ocre[col].apply(lambda x: x.decode('utf-8') if type(x) is not float else x)

In [ ]:
cleaned_ocre.tail()

# Store Data

In [5]:
cnx = sqlite3.connect('../Data/'+title+'.sqlite')
cur = cnx.cursor()

In [6]:
cur.execute('DROP TABLE IF EXISTS allData')
cur.execute('''
CREATE TABLE allData (
  authority text,
  material text,
  denomination text,
  subject text,
  mint text,
  date text,
  description text,
  obverseType text,
  reverseType text,
  startDate int,
  endDate int,
  source text,
  url text
);
''')

In [ ]:
cur.execute('DROP TABLE IF EXISTS britishMuseum')
cur.execute('''CREATE TABLE britishMuseum(associatedNames TEXT, authority TEXT, bibliography TEXT, culture TEXT, 
                    curatorComment TEXT, dates TEXT, denomination TEXT, description TEXT, material TEXT, 
                    museumNumber TEXT, objectType TEXT, obverseLegend TEXT, obverseType TEXT, mint TEXT, 
                    reverseLegend TEXT, reverseType TEXT, state TEXT, subjects TEXT, weight INT, url TEXT, 
                    source TEXT, startDate INT, endDate INT)''')

In [ ]:
cur.executemany("""INSERT INTO britishMuseum (associatedNames, authority, bibliography, culture,
                        curatorComment, dates, denomination, description,
                        material, museumNumber, objectType, obverseLegend, obverseType,
                        mint, reverseLegend, reverseType, state, subjects, weight,
                        url, source, startDate, endDate) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                list(cleaned_bm.to_records(index=False)))

In [7]:
cur.execute('''
INSERT INTO allData (authority, material, denomination, subject, mint, date, description, 
                        obverseType, reverseType, startDate, endDate, source, url)
SELECT
  authority,
  material,
  denomination,
  subjects,
  mint,
  dates,
  description,
  obverseType,
  reverseType,
  startDate,
  endDate,
  source,
  url
FROM britishMuseum;
''')

In [ ]:
cur.execute('DROP TABLE IF EXISTS americanNumismaticSociety')
cur.execute('''CREATE TABLE americanNumismaticSociety(authority TEXT, deity TEXT, denomination TEXT, description TEXT, 
                       issuer TEXT, material TEXT, mint TEXT, obverseLegend TEXT, obverseType TEXT, portrait TEXT, 
                       reference TEXT, reverseLegend TEXT, reverseType TEXT, url TEXT, dates TEXT, source TEXT, 
                       startDate INT, endDate INT)''')

In [ ]:
cur.executemany("""INSERT INTO americanNumismaticSociety (authority, deity, denomination, description, 
                       issuer, material, mint, obverseLegend, obverseType, portrait, reference, reverseLegend, 
                       reverseType, url, dates, source, startDate, endDate) 
                    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                list(cleaned_ans.to_records(index=False)))

In [8]:
cur.execute('''
INSERT INTO allData (authority, material, denomination, subject, mint, date, 
                        description, obverseType, reverseType, startDate, endDate, source, url)
SELECT
  authority,
  material,
  denomination,
  portrait,
  mint,
  dates,
  description,
  obverseType,
  reverseType,
  startDate,
  endDate,
  source,
  url
FROM americanNumismaticSociety;
''')

In [ ]:
cur.execute('DROP TABLE IF EXISTS OCRE')
cur.execute('''CREATE TABLE OCRE(authority TEXT, deity TEXT, denomination TEXT, description TEXT, 
                       issuer TEXT, material TEXT, mint TEXT, obverseLegend TEXT, obverseType TEXT, portrait TEXT, 
                       reference TEXT, reverseLegend TEXT, reverseType TEXT, url TEXT, dates TEXT, source TEXT, 
                       startDate INT, endDate INT)''')

In [ ]:
cur.executemany("""INSERT INTO OCRE (authority, deity, denomination, description, 
                       issuer, material, mint, obverseLegend, obverseType, portrait, reference, reverseLegend, 
                       reverseType, url, dates, source, startDate, endDate) 
                    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                list(cleaned_ans.to_records(index=False)))

In [9]:
cur.execute('''
INSERT INTO allData (authority, material, denomination, subject, mint, date, 
                        description, obverseType, reverseType, startDate, endDate, source, url)
SELECT
  authority,
  material,
  denomination,
  portrait,
  mint,
  dates,
  description,
  obverseType,
  reverseType,
  startDate,
  endDate,
  source,
  url
FROM OCRE;
''')

In [10]:
cnx.commit()